这是transform的手撕+code练习

```python
import torch.nn.init as init
init.kaiming_normal_(weights, mode='fan_in', nonlinearity='relu')
init.xavier_normal_(input_,output_)

nn.softmax就是网络里的一部分，而f.softmax只是一个函数

所以区分一下torch.nn和torch.nn.functional

torch.randn(input_,output_) #(0,1)
torch.normal(mu,var,size=(input_,output_))
f.relu()
f.softmax()
f.linear()
```

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

/Users/chenhaoyang/miniconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
X=torch.randn(128,64,512)  #batch, time, dimension
print(X.shape)
d_model=512
n_head=8

In [ ]:
class multi_head_attention(nn.Module):  #需要继承nn.Module库
    def __init__(self,d_model,n_head)->None:
        super(multi_head_attention, self).__init__()
        self.d_model=d_model
        self.n_head=n_head
        
        self.w_q=nn.Linear(d_model,d_model)
        self.w_k=nn.Linear(d_model,d_model)
        self.w_v=nn.Linear(d_model,d_model)
        self.fc_out=nn.Linear(d_model,d_model)
        self.softmax=nn.Softmax(dim=-1)  #dim=-1只对最后一个维度进行操作
        
    def forward(self,x):
        batch,time,dimension=x.shape
        n_d=self.d_model//self.n_head# 每个head的维度
        Q=self.w_q(x)
        K=self.w_k(x)
        V=self.w_v(x)
        
        Q=Q.view(batch,time,self.n_head,n_d).permute(0,2,1,3)
        K=K.view(batch,time,self.n_head,n_d).permute(0,2,1,3)
        V=V.view(batch,time,self.n_head,n_d).permute(0,2,1,3)
        
        A=(K.T).dot(Q)/nn.sqrt(self.d_model)
        A_hat=self.softmax(A)
        B=A_hat.dot(V)  
        return B      
        
        

定义一个自定义的神经网络模块时，通常会继承 torch.nn.Module

nn.Module 是 PyTorch 中所有神经

可以通过torch.save()和torch.load()保存恢复模型
继承 nn.Module 后，你可以通过以下方式扩展它
* 定义自定义的网络结构（子模块）。
* 定义自定义的前向传播逻辑（forward 方法）。

这是 multi_head_attention 类的初始化方法，用于定义该模块的属性和子模块

`def __init__(self,) -> None:`

除了 self 参数外，你可以添加其他自定义的参数。这些参数可以用来配置类的行为、初始化特定的属性或子模块。

super().__init__() 是一个关键步骤，用于调用父类（nn.Module）的初始化方法：

`super(multi_head_attention, self).__init__()`

如果没有调用 super().__init__()，nn.Module 的一些功能（如参数管理、设备分配）将无法正常工作。

GPT的multi-head Attention

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout=0.1):
        """
        初始化多头注意力模块。
        
        Args:
        - embed_dim (int): 输入的嵌入维度（d_model）。
        - num_heads (int): 注意力头的数量。
        - dropout (float): Dropout 的比例。
        """
        super(MultiHeadAttention, self).__init__()
        
        # 验证 embed_dim 是否可以被 num_heads 整除
        assert embed_dim % num_heads == 0, "embed_dim 必须能够整除 num_heads"
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads  # 每个头的维度 d_k
        
        # 定义 Q, K, V 的权重
        self.W_Q = nn.Linear(embed_dim, embed_dim)  # (embed_dim, embed_dim)
        self.W_K = nn.Linear(embed_dim, embed_dim)  # (embed_dim, embed_dim)
        self.W_V = nn.Linear(embed_dim, embed_dim)  # (embed_dim, embed_dim)
        
        # 输出权重
        self.fc_out = nn.Linear(embed_dim, embed_dim)  # (embed_dim, embed_dim)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        """
        前向传播。
        
        Args:
        - x (Tensor): 输入张量，形状为 (batch_size, seq_length, embed_dim)。
        
        Returns:
        - out (Tensor): 输出张量，形状为 (batch_size, seq_length, embed_dim)。
        - attention (Tensor): 注意力矩阵，形状为 (batch_size, num_heads, seq_length, seq_length)。
        """
        batch_size, seq_length, embed_dim = x.size()
        
        # 计算 Q, K, V
        Q = self.W_Q(x)  # (batch_size, seq_length, embed_dim)
        K = self.W_K(x)  # (batch_size, seq_length, embed_dim)
        V = self.W_V(x)  # (batch_size, seq_length, embed_dim)
        
        # 拆分为多头
        Q = Q.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)  # (batch_size, num_heads, seq_length, head_dim)
        K = K.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)  # (batch_size, num_heads, seq_length, head_dim)
        V = V.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)  # (batch_size, num_heads, seq_length, head_dim)
        
        # 点积注意力计算
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.head_dim ** 0.5)  # (batch_size, num_heads, seq_length, seq_length)
        attention = torch.softmax(attention_scores, dim=-1)  # 归一化
        attention = self.dropout(attention)  # Dropout
        
        # 加权 V
        out = torch.matmul(attention, V)  # (batch_size, num_heads, seq_length, head_dim)
        
        # 合并多头
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_length, embed_dim)  # (batch_size, seq_length, embed_dim)
        
        # 输出线性变换
        out = self.fc_out(out)  # (batch_size, seq_length, embed_dim)
        
        return out, attention